In [ ]:
# Настройка ожиданий
# Как работают методы get и find_element

# Разберем еще один простой тест на WebDriver, проверяющий работу кнопки.
# Тестовый сценарий выглядит так:

# Открыть страницу http://suninjuly.github.io/wait1.html
# Нажать на кнопку "Verify"
# Проверить, что появилась надпись "Verification was successful!"

In [ ]:
# Для открытия страницы мы используем метод get,
# Для открытия страницы мы используем метод get,
# затем находим нужную кнопку с помощью одного из методов find_element_by_ и нажимаем на нее с помощью метода click.
# Далее находим новый элемент с текстом и проверяем соответствие текста на странице ожидаемому тексту.

# Вот как выглядит код автотеста:

from selenium import webdriver

browser = webdriver.Chrome()
browser.get("http://suninjuly.github.io/wait1.html")

button = browser.find_element_by_id("verify")
button.click()
message = browser.find_element_by_id("verify_message")

assert "successful" in message.text

# Попробуйте сначала выполнить тест вручную, а затем запустить автотест. В первом случае, вы завершите тест успешно,
# во втором случае автотест упадет с сообщением NoSuchElementException для элемента c id="verify". Почему так происходит?

# Команды в Python выполняются синхронно, то есть, строго последовательно.
# Пока не завершится команда get, не начнется поиск кнопки. Пока кнопка не найдена, не будет сделан клик по кнопке и так далее.

In [ ]:
# Пример показался кривым:
--------Вот рабочая заготовка, !НО со "sleep", тест "падать не будет":----------------

from selenium import webdriver
import time


try: 
    browser = webdriver.Chrome()
    browser.get("http://suninjuly.github.io/wait1.html")

    time.sleep(2)

    button = browser.find_element_by_id("verify")
    button.click()
    message = browser.find_element_by_id("verify_message")
    static_message = "Verification was successful!"
   

    assert static_message == message.text, "Сообщение в алерте не то, что надо"
    
finally:
    time.sleep(2)
    browser.quit()


In [ ]:
# Давайте быстрее это починим: time.sleep()
# Теперь, когда мы уже знаем, что кнопка появляется с задержкой, мы можем добавить паузу до начала поиска элемента.
# Мы уже использовали библиотеку time ранее. Давайте применим ее и сейчас:

from selenium import webdriver
import time

browser = webdriver.Chrome()
browser.get("http://suninjuly.github.io/wait1.html")

time.sleep(1)
button = browser.find_element_by_id("verify")
button.click()
message = browser.find_element_by_id("verify_message")

assert "successful" in message.text

In [ ]:
# Есть способы получше: Selenium Waits (Implicit Waits)
# Надеемся, вы поняли, что решение с time.sleep() плохое: оно не масштабируемое и трудно поддерживаемое.

# В Selenium WebDriver есть специальный способ организации такого ожидания, который позволяет задать ожидание
# при инициализации драйвера, чтобы применить его ко всем тестам.
# Ожидание называется неявным (Implicit wait), так как его не надо явно указывать каждый раз,
# когда мы выполняем поиск элементов, оно автоматически будет применяться при вызове каждой последующей команды.

In [ ]:
# Улучшим наш тест с помощью неявных ожиданий. Для этого нам нужно будет убрать time.sleep()
# и добавить одну строчку с методом implicitly wait:

from selenium import webdriver

browser = webdriver.Chrome()

# говорим WebDriver искать каждый элемент в течение 5 секунд
browser.implicitly_wait(5)

browser.get("http://suninjuly.github.io/wait1.html")

button = browser.find_element_by_id("verify")
button.click()
message = browser.find_element_by_id("verify_message")

assert "successful" in message.text

# Теперь мы можем быть уверены, что при небольших задержках в работе сайта наши тесты продолжат работать стабильно.
# На каждый вызов команды find_element WebDriver будет ждать 5 секунд до появления элемента на странице прежде,
# чем выбросить исключение NoSuchElementException.

In [ ]:
# lesson4_steps6 - Задание: Про Exceptions
# Спрашивают в комментариях про смысл задания.
# Смысл в том, чтобы запустив код (например мой вариант), увидеть в консоли как отработает код и выбросит
# исключение: NoSuchElementException. Результатом решения задачи, было увидеть данное сообщение и выбрать его из предложенных.

from selenium import webdriver

browser = webdriver.Chrome()
browser.implicitly_wait(5)
browser.get("http://suninjuly.github.io/cats.html")

browser.find_element_by_id("button")
button.click()

assert "successful" in message.text

time.sleep(7)
browser.quit()

In [ ]:
# Explicit Waits (WebDriverWait и expected_conditions)
# В предыдущем шаге мы решили проблему с ожиданием элементов на странице.
# Однако методы find_element проверяют только то, что элемент появился на странице.
# В то же время элемент может иметь дополнительные свойства, которые могут быть важны для наших тестов.

# Рассмотрим пример с кнопкой, которая отправляет данные:

# Кнопка может быть неактивной, то есть её нельзя кликнуть;
# Кнопка может содержать текст, который меняется в зависимости от действий пользователя.
#       Например, текст "Отправить" после нажатия кнопки поменяется на "Отправлено";
# Кнопка может быть перекрыта каким-то другим элементом или быть невидимой.

In [ ]:
# Если мы хотим в тесте кликнуть на кнопку, а она в этот момент неактивна,
# то WebDriver все равно проэмулирует действие нажатия на кнопку, но данные не будут отправлены.

# Давайте попробуем запустить следующий тест:

from selenium import webdriver

browser = webdriver.Chrome()
# говорим WebDriver ждать все элементы в течение 5 секунд
browser.implicitly_wait(5)

browser.get("http://suninjuly.github.io/wait2.html")

button = browser.find_element_by_id("verify")
button.click()
message = browser.find_element_by_id("verify_message")

assert "successful" in message.text

# Мы видим, что WebDriver смог найти кнопку с id="verify" и кликнуть по ней,
# но тест упал на поиске элемента "verify_message" с итоговым сообщением:

no such element: Unable to locate element: {"method":"id","selector":"verify_message"}


In [ ]:
# Чтобы тест был надежным, нам нужно не только найти кнопку на странице, но и дождаться, когда кнопка станет кликабельной.
# Для реализации подобных ожиданий в Selenium WebDriver существует понятие явных ожиданий (Explicit Waits),
# которые позволяют задать специальное ожидание для конкретного элемента.
# Задание явных ожиданий реализуется с помощью инструментов WebDriverWait и expected_conditions. Улучшим наш тест:

from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium import webdriver

browser = webdriver.Chrome()

browser.get("http://suninjuly.github.io/wait2.html")

# говорим Selenium проверять в течение 5 секунд, пока кнопка не станет кликабельной
button = WebDriverWait(browser, 5).until(
        EC.element_to_be_clickable((By.ID, "verify"))
    )
button.click()
message = browser.find_element_by_id("verify_message")

assert "successful" in message.text

In [ ]:
# Если мы захотим проверять, что кнопка становится неактивной после отправки данных,
# то можно задать негативное правило с помощью метода until_not:

# говорим Selenium проверять в течение 5 секунд пока кнопка станет неактивной
button = WebDriverWait(browser, 5).until_not(
        EC.element_to_be_clickable((By.ID, "verify"))
    )

In [ ]:
# lesson4_steps8 - Задание: ждем нужный текст на странице

# Попробуем теперь написать программу, которая будет бронировать нам дом для отдыха по строго заданной цене.
# Более высокая цена нас не устраивает, а по более низкой цене объект успеет забронировать кто-то другой.

# В этой задаче вам нужно написать программу, которая будет выполнять следующий сценарий:

# Открыть страницу http://suninjuly.github.io/explicit_wait2.html
# Дождаться, когда цена дома уменьшится до $100 (ожидание нужно установить не меньше 12 секунд)
# Нажать на кнопку "Book"
# Решить уже известную нам математическую задачу (используйте ранее написанный код) и отправить решение
# Чтобы определить момент, когда цена аренды уменьшится до $100, используйте метод
# text_to_be_present_in_element из библиотеки expected_conditions.

In [ ]:
# lesson4_steps8 - Задание: ждем нужный текст на странице
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium import webdriver
import time
import math

# Считает формулу на странице
def calc(x):
    return str(math.log(abs(12*math.sin(int(x)))))

try:
    # Загружаем браузер, открываем страницу https://...
    link = "http://suninjuly.github.io/explicit_wait2.html"
    browser = webdriver.Chrome()
    browser.get(link)
    
    # говорим Selenium проверять в течение 12 секунд, пока цена не станет = $100
    WebDriverWait(browser, 12).until(EC.text_to_be_present_in_element((By.ID, "price"), "$100"))

    # Находим кнопку "Book" и жмём на неё.
    button_book = browser.find_element_by_id("book")
    button_book.click()

    # Скроллим документ вниз до тех пор, пока не будет видна кнопка отправки формы
    button_submit = browser.find_element_by_xpath('//button[@type="submit"]')
    browser.execute_script("return arguments[0].scrollIntoView(true);", button_submit)
    
    x_element = browser.find_element_by_id("input_value")   #находит переменную
    x = x_element.text                                      #присваивает ее за х
    y = calc(x)                                             #высчитывает формулу с х
    print('f(', x, ') = ', y)

    #находит поле для ввода ответа
    input_result = browser.find_element_by_xpath('//input[@type="text"]') 
    input_result.send_keys(y)
    
    #Нажать на кнопку "Отправить".
    button_submit.click()
    
    print("Тест успешно завершен. 20 сек на закрытие браузера...")
    
except Exception as error:
    print(f'Произошла ошибка, вот её трэйсбэк: {error}')

    
finally:
    alert=browser.switch_to_alert() # скрапим всплывающее сообщение
    print('---------------------------------\n\n\n\n',alert.text)# выводим его в консоль
    # успеваем скопировать код за 30 секунд
    time.sleep(20)
    # закрываем браузер после всех манипуляций
    browser.close()
    browser.quit()

# не забываем оставить пустую строку в конце файла

In [ ]:
# lesson4_steps8 - Задание: ждем нужный текст на странице
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from math import log, sin
import pyperclip

browser = webdriver.Chrome()
browser.get('http://suninjuly.github.io/explicit_wait2.html')

# Дождаться, когда цена дома уменьшится до 10000 RUR
WebDriverWait(browser, 13).until(EC.text_to_be_present_in_element((By.ID, 'price'),'10000 RUR'))

# Нажать на кнопку "Забронировать"
browser.find_element_by_id('book').click()

# Решить уже известную нам математическую задачу (используйте ранее написанный код)
browser.find_element_by_id('answer').send_keys(
  str(log(abs(12*sin(int(browser.find_element_by_id('input_value').text)))))
)
browser.find_element_by_id('solve').click()

# Копирование числа из алерта в буфер обмена
alert = browser.switch_to.alert
alert_text = alert.text
addToClipBoard = alert_text.split(': ')[-1]
pyperclip.copy(addToClipBoard)

In [ ]:
# lesson4_steps8 - Задание: ждем нужный текст на странице
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium import webdriver
import math


def calc(x):
    return str(math.log(abs(12 * math.sin(int(x)))))


browser = webdriver.Chrome()
link = 'http://suninjuly.github.io/explicit_wait2.html'
browser.implicitly_wait(5)  # ожидание действия в 5 секунд, если не нашелся selector
try:
    browser.get(link)
    WebDriverWait(browser, 15).until(EC.text_to_be_present_in_element((By.ID, "price"), '100'))
    browser.find_element_by_css_selector('#book').click()
    button = browser.find_element_by_css_selector('#solve')
    browser.execute_script("return arguments[0].scrollIntoView(true);", button)
    x1 = browser.find_element_by_css_selector('#input_value').text
    y = calc(x1)
    browser.find_element_by_css_selector('#answer').send_keys(y)
    button.click()
    alert = browser.switch_to.alert
    answer = alert.text.split()[-1]
    print(answer)
    alert.accept()
    # авторизуемся на Степике
    browser.get('https://stepik.org/catalog?auth=login&language=ru')

    browser.find_element_by_id('id_login_email').send_keys('login')  # здесь вводится e-mail
    browser.find_element_by_id('id_login_password').send_keys('password')  # здесь вводится пароль

    browser.find_element_by_class_name('sign-form__btn').click()
    browser.get('https://stepik.org/lesson/181384/step/8?unit=156009')

    answer_input = browser.find_element_by_css_selector('textarea')
    browser.execute_script("return arguments[0].scrollIntoView(true);", answer_input)
    answer_input.send_keys(answer)

    button = browser.find_element_by_class_name('submit-submission')
    browser.execute_script("return arguments[0].scrollIntoView(true);", button)
    button.click()
finally:
    browser.quit()

In [ ]:
# lesson4_steps8 - Задание: ждем нужный текст на странице
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium import webdriver
import math


def calc(x):
    return str(math.log(abs(12*math.sin(int(x)))))


try:
    # 1. Открыть страницу по URL.
    link = "http://suninjuly.github.io/explicit_wait2.html"
    browser = webdriver.Chrome()
    browser.get(link)

    # 2. Дождаться, когда цена дома уменьшится до $100 (ожидание нужно установить не меньше 12 секунд)
    price = WebDriverWait(browser, 15).until(
        EC.text_to_be_present_in_element((By.ID, "price"), "$100")
    )

    # 3. Нажать на кнопку "Book"
    button1 = browser.find_element_by_id("book")
    button1.click()

    # 4. Решить уже известную нам математическую задачу (используйте ранее написанный код) и отправить решение
    x_element = browser.find_element_by_id("input_value")
    x = x_element.text
    y = calc(x)

    input1 = browser.find_element_by_id("answer")
    input1.send_keys(y)

    button2 = browser.find_element_by_id("solve")
    button2.click()

    print(browser.switch_to.alert.text)

finally:
    browser.quit()

In [ ]:
# lesson4_steps8 - Задание: ждем нужный текст на странице
"""
1. Открыть страницу http://suninjuly.github.io/explicit_wait2.html
2. Дождаться, когда цена дома уменьшится до 10000 RUR (ожидание нужно установить не меньше 12 секунд)
3. Нажать на кнопку "Забронировать"
4. Решить уже известную нам математическую задачу (используйте ранее написанный код) и отправить решение
"""

from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import math

link = 'http://suninjuly.github.io/explicit_wait2.html'


def calc(x):
    """
    Функция возращает результат формулы ln(abs(12*sin(x)))
    :param x:
    :return str:
    """
    return str(math.log(abs(12 * math.sin(int(x)))))


driver = webdriver.Chrome()
driver.get(link)

# Ожидаем когда текст внутри элемента == '10000 RUR'
# Время ожидания 12 секунд
WebDriverWait(driver, 12).until(
    EC.text_to_be_present_in_element((By.CSS_SELECTOR, '#price'), '10000 RUR')
)

# Нажимаем кнопку "Забронировать"
driver.find_element_by_css_selector('#book').click()

# Получаем строковое значение x
x = driver.find_element_by_css_selector('#input_value').text

# Получаем результат формулы
result = calc(x)

# Передаем результат в поле ввода
driver.find_element_by_css_selector('#answer').send_keys(result)

# Нажимаем кнопку "Отправить"
driver.find_element_by_css_selector('#solve').click()

In [ ]:
# lesson4_steps8 - Задание: ждем нужный текст на странице
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium import webdriver
import pyperclip

import math
import time


def calc(x):
    """
    Function realized mathematical formula
    :param x: str
    :return: int
    """
    return str(math.log(abs(12 * math.sin(int(x)))))

try:
    # make a link
    link = "http://suninjuly.github.io/explicit_wait2.html"
    #
    browser = webdriver.Chrome()
    # move to page
    browser.get(link)

    # Waiting when hose price will be 100
    actual_price = WebDriverWait(browser, 12).until(
        EC.text_to_be_present_in_element((By.ID, "price"), '100'))

    # check house price
    if actual_price:
        # if price == 100 find the button and click it
        button = browser.find_element_by_id('book').click()

    # we must scroll down our page and we do it
    button = browser.find_element_by_id('solve')
    browser.execute_script("return arguments[0].scrollIntoView(true);", button)

    # we must read the number for our mathematical expression and send them
    nowrap = browser.find_element_by_id('input_value').text
    expression_result = calc(nowrap)

    # we must find the line for our answer and put our answer there
    output_answer = browser.find_element_by_name("text")
    output_answer.send_keys(expression_result)

    button.click()

    # we want copy the alert answer to buffer
    alert = browser.switch_to.alert
    alert_text = alert.text
    add_to_clip_board = alert_text.split(': ')[-1]
    pyperclip.copy(add_to_clip_board)

finally:
    time.sleep(5)
    browser.quit()

In [ ]:
# lesson4_steps8 - Задание: ждем нужный текст на странице
# Чтоб работал модуль  pyperclip, который копирует код ответа в буфер обмена, нужно его установить в виртуальном окружении:
# (selenium_env) C:\Users\users_name>pip install pyperclip

from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import math
import pyperclip

def calc(x):
  return str(math.log(abs(12*math.sin(int(x)))))
browser = webdriver.Chrome()
browser.get("http://suninjuly.github.io/explicit_wait2.html")
try:
    price = WebDriverWait(browser, 15).until(EC.text_to_be_present_in_element((By.ID, "price"), "$100"))
    button1 = browser.find_element_by_id("book").click()
    input1 = browser.find_element_by_id('answer').send_keys(calc(browser.find_element_by_id('input_value').text))
    button2 = browser.find_element_by_id("solve").click()
    alert = browser.switch_to.alert
    alert_text = alert.text
    addToClipBoard = alert_text.split(': ')[-1]
    pyperclip.copy(addToClipBoard)
except Exception as error:
    print(f'Произошла ошибка, вот её описание: {error}')
finally:
    browser.quit()

In [ ]:
# lesson4_steps8 - Задание: ждем нужный текст на странице
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as ES
import math


def calc(x):
    return str(math.log(abs(12 * math.sin(int(x)))))

try:
    browser = webdriver.Chrome()
    browser.get('http://suninjuly.github.io/explicit_wait2.html')

    WebDriverWait(browser, 12).until(
        ES.text_to_be_present_in_element((By.ID, 'price'), '100')
    )
    button = browser.find_element(By.ID, 'book').click()
    browser.execute_script("window.scrollTo(0, 99)")
    x = browser.find_element(By.ID, 'input_value').text
    browser.find_element(By.ID, 'answer').send_keys(calc(x))
    browser.find_element(By.ID, 'solve').click()
    print(browser.switch_to.alert.text.split()[-1])
finally:
    browser.quit()

In [ ]:
# lesson4_steps8 - Задание: ждем нужный текст на странице
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium import webdriver

browser = webdriver.Chrome()
# открыть страницу http://suninjuly.github.io/explicit_wait2.html
browser.get("http://suninjuly.github.io/explicit_wait2.html")

# Дождаться, когда цена дома уменьшится до $100 (ожидание нужно установить не меньше 12 секунд)
WebDriverWait(browser, 12).until(
    EC.text_to_be_present_in_element((By.ID, "price"), "$100")
    )
browser.find_element_by_css_selector("button.btn").click()
# нажать на кнопку "Book"

# решить уже известную нам математическую задачу (используйте ранее написанный код) и отправить решение
import math
import time

def calc(x):
    return str(math.log(abs(12 * math.sin(int(x)))))

x_element = browser.find_element_by_id("input_value")
x = x_element.text
y = calc(x)
# считать значение для переменной x

browser.find_element_by_id("answer").send_keys(y)
# вводим текст полученного числа

# говорим Selenium проверять в течение 5 секунд, пока кнопка не станет кликабельной
button = WebDriverWait(browser, 5).until(
        EC.element_to_be_clickable((By.ID, "solve"))
    )
button.click()
# нажать на кнопку "Submit!"

time.sleep(5)

browser.quit()

In [ ]:
# lesson5_steps2 - Выберите все подходящие ответы из списка
#Логинимся
browser.find_element_by_id("id_login_email").send_keys('email')
browser.find_element_by_id("id_login_password").send_keys('pass')
browser.find_element_by_css_selector(".sign-form__btn").click()
 
#переходим на страницу теста
browser.get('https://stepik.org/lesson/236205/step/2?unit=208637')
 
#Кликаем чек-боксы отправляем результат
browser.find_element_by_css_selector(".ember-view[data-type='choice-quiz'] .s-checkbox:nth-child(1)").click()
browser.find_element_by_css_selector(".ember-view[data-type='choice-quiz'] .s-checkbox:nth-child(2)").click()
browser.find_element_by_css_selector(".ember-view[data-type='choice-quiz'] .s-checkbox:nth-child(3)").click()
browser.find_element_by_css_selector(".ember-view[data-type='choice-quiz'] .s-checkbox:nth-child(4)").click()
browser.find_element_by_class_name("submit-submission").click()
 
#Тест выполнен неверно что 99%, делаем условие и цикл
browser.find_element_by_class_name("again-btn").text == 'Попробовать снова'
browser.find_element_by_class_name("again-btn").click()

In [ ]:
# 


In [ ]:
# 
